In [1]:
from Liputan6 import Liputan6 #class news Scrapper
from datetime import timedelta, date, datetime
from IPython.display import clear_output

In [2]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [3]:
# health
# start_date = date(2018, 3, 8)
obj = Liputan6()
end_date = date(2018, 9, 1)
start_date = date(2018, 8, 11)
init_page = 2
list_cat = [['news', 'news'], ['otomotif', 'otomotif'], ['properti', 'properti'],['bisnis','bisnis'],['showbiz','showbiz'],
           ['bola','bola'],['tekno','tekno'],['fashion-beauty','fashion-beauty'],['lifestye','lifestye']]
init_cat = list_cat[3:]

In [ ]:
for single_date in daterange(start_date, end_date):
    for cat in init_cat:
        clear_output()
        print(single_date.strftime("%Y-%m-%d"))
        print(cat[0])
        obj.getAllBerita([], init_page, cat[0], cat[1], single_date.strftime("%Y/%m/%d"))
        init_page = 1
    init_cat = list_cat

2018-08-11
bola
page  1
https://www.liputan6.com/bola/indeks/2018/08/11?page=1
Insert berita  Komentar Presiden Jokowi Setelah Timnas Indonesia Raih Juara Piala AFF U-16 2018.
masuk
Insert berita  Bagus Kahfi Jadi Top Skorer di Pentas Piala AFF U-16 2018
masuk
Insert berita  Sirkuit Sentul Bakal Gelar Balapan Malam untuk Pertama Kali
masuk
